In [1]:
import numpy as np
import os
import pandas as pd
from helpers import create_polling_id
from glob import glob

os.chdir("../")

In [2]:
year = 2021
election_round = "senate" # first_round, runoff

In [3]:
if election_round in ["first_round", "runoff"]:
    if year == 2013:
        output = []
        for file in glob(f"data/Chile/{year}_{election_round}_*.xlsx"):
            region_id = file.replace(f"data/Chile\{year}_{election_round}_", "")[0:2]
            tmp = pd.read_excel(file)
            tmp["region_id"] = region_id
            output.append(tmp)

        df = pd.concat(output, ignore_index=True)
    elif year in [2017, 2021]:
        df = pd.read_excel(f"data/Chile/{year}_{election_round}.xlsx")

    df = df.rename(columns={
        "Candidato": "candidate", 
        "Votos TRICEL": "value", 
        "Nro. Mesa": "mesa", 
        "Tipo de mesa": "tipo mesa",
        "Circ.Electoral": "circ. electoral"
    })
    df["candidate"] = df["candidate"].str.upper().str.strip()
    df["value"] = df["value"].fillna(0)

    df.columns = [x.lower().strip() for x in df.columns]

    df.loc[df["provincia"] == "ÑUBLE", "region_id"] = 16
    df.loc[df["provincia"] == "ÑUBLE", "región"] = 16

    df = df.dropna(subset=["comuna"])

    def create_polling_id(data, columns=[]):
        output = data[columns[0]].fillna("###").copy()

        for column in columns[1:]:
            output += "-" + data[column].fillna("###").astype(str)
        return output

    if year == 2013:
        df["polling_id"] = create_polling_id(df, columns=["comuna", "circ. electoral", "mesa", "tipo mesa"])

    elif year in [2017, 2021]:
        df["polling_id"] = create_polling_id(df, columns=["comuna", "circ. electoral", "mesa", "tipo mesa", "mesas fusionadas"])

elif election_round in ["senate"]:
    output = []
    for file in glob(f"data/Chile/{year}/{election_round}/*.xlsx"):
        tmp = pd.read_excel(file)
        output.append(tmp)

    df = pd.concat(output, ignore_index=True)
    df = df.dropna(subset=["Nro. Región"])

    df = df.rename(columns={
        "Candidato": "candidate", 
        "Votos TRICEL": "value", 
        "Nro. Mesa": "mesa", 
        "Tipo de mesa": "tipo mesa",
        "Circ.Electoral": "circ. electoral",
        "Partido": "party"
    })
    df.columns = [x.lower().strip() for x in df.columns]

    for col in ["nro. región", "circ. senatorial", "distrito"]:
        df[col] = df[col].astype(int)

    df["polling_id"] = create_polling_id(df, columns=["circ. senatorial", "distrito", "comuna", "circ. electoral", "local", "mesas fusionadas"])


df["candidate"] = df["candidate"].replace({
    "VOTOS EN BLANCO": "BLANK",
    "VOTOS BLANCOS": "BLANK",
    "VOTOS NULOS": "SPOILT"
})

df.head()

,nro. región,región,provincia,circ. senatorial,distrito,comuna,circ. electoral,local,mesa,tipo mesa,mesas fusionadas,electores,nro. en voto,lista,pacto,party,candidate,value,polling_id
0,10,DE LOS LAGOS,CHILOE,13,26,ANCUD,ANCUD,COLEGIO EL PILAR EDUCACION BASICA,28.0,V,28V,319.0,10.0,AA,CHILE PODEMOS +,UNION DEMOCRATA INDEPENDIENTE,IVAN MOREIRA BARROS,15.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...
1,10,DE LOS LAGOS,CHILOE,13,26,ANCUD,ANCUD,COLEGIO EL PILAR EDUCACION BASICA,28.0,V,28V,319.0,11.0,AA,CHILE PODEMOS +,UNION DEMOCRATA INDEPENDIENTE,JAVIER HERNANDEZ HERNANDEZ,0.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...
2,10,DE LOS LAGOS,CHILOE,13,26,ANCUD,ANCUD,COLEGIO EL PILAR EDUCACION BASICA,28.0,V,28V,319.0,12.0,AA,CHILE PODEMOS +,RENOVACION NACIONAL,ALEJANDRO SANTANA TIRACHINI,24.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...
3,10,DE LOS LAGOS,CHILOE,13,26,ANCUD,ANCUD,COLEGIO EL PILAR EDUCACION BASICA,28.0,V,28V,319.0,13.0,AA,CHILE PODEMOS +,RENOVACION NACIONAL,CARLOS IGNACIO KUSCHEL SILVA,2.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...
4,10,DE LOS LAGOS,CHILOE,13,26,ANCUD,ANCUD,COLEGIO EL PILAR EDUCACION BASICA,28.0,V,28V,319.0,14.0,AB,NaN,PARTIDO DE LA GENTE,CRISTIAN ARNOLDO ALVAREZ MANSILLA,5.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...


#### Abstentions
The election data from 2017 and 2021 includes as a feature the number of expected electors by polling station. We flag the data, by including "Abstentions" as a candidate.

In [4]:
df_abstentions = pd.DataFrame()
if year in [2017, 2021] and election_round:
    df_abstentions = df.groupby("polling_id").agg({"value": "sum"}).reset_index()
    df_abstentions = pd.merge(
        df[["polling_id", "electores"]].drop_duplicates(), 
        df_abstentions, 
        on="polling_id"
    )
    df_abstentions["value2"] = df_abstentions["electores"] - df_abstentions["value"]
    df_abstentions = df_abstentions[["polling_id", "value2"]].rename(columns={"value2": "value"})
    df_abstentions["candidate"] = "ABSTENTION"
    df_abstentions["flag_candidates"] = 0

#### Spoilt votes

In [5]:
df_spoilt = df[df["candidate"].isin(["SPOILT", "BLANK"])].copy()
df_spoilt = df_spoilt[["candidate", "value", "polling_id"]]
df_spoilt["flag_candidates"] = 0
df_spoilt.head()

,candidate,value,polling_id,flag_candidates
18,SPOILT,8.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...,0
19,BLANK,16.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...,0
38,SPOILT,8.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...,0
39,BLANK,7.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...,0
58,SPOILT,4.0,13-26-ANCUD-ANCUD-COLEGIO EL PILAR EDUCACION B...,0


In [6]:
df.candidate.unique()

array(['IVAN MOREIRA BARROS', 'JAVIER HERNANDEZ HERNANDEZ',
       'ALEJANDRO SANTANA TIRACHINI', 'CARLOS IGNACIO KUSCHEL SILVA',
       'CRISTIAN ARNOLDO ALVAREZ MANSILLA',
       'CAROLINA SILVANA FIGUEROA OLAVE',
       'ANGELICA MARIA HENRIQUEZ FERNANDEZ',
       'CHRISTIAN ANGEL MAURICIO CID BARAHONA',
       'RABINDRANATH QUINTEROS LARA', 'FIDEL ESPINOZA SANDOVAL',
       'PAMELA SOLEDAD BERTIN HERNANDEZ', 'JORGE BRUNO KEIM GUTIERREZ',
       'MAURICIO MARTINEZ HURTADO', 'ORIETTA ELIANA LLAUCA HUALA',
       'FERNANDA HEDERRA WILLIAMS', 'PAOLA ALEJANDRA VENEGAS ARRIAGADA',
       'NATALIA RAVANALES TORO', 'FABIOLA MOYA SANDOVAL', 'SPOILT',
       'BLANK', 'JUAN JOSE CLAUDIO ARCOS SRDANOVIC',
       'SANDRA AMAR MANCILLA', 'ALEJANDRO JUAN KUSANOVIC GLUSEVIC',
       'JESUS RODRIGO GUTIERREZ OLIVARES',
       'CARLA DANIELA AMTHAUER GONZALEZ', 'NINEN GOMEZ VILLEGAS',
       'MARIA DE LOS ANGELES FLORES RODRIGUEZ',
       'JUAN DEL KARANO MARQUEZ BORQUEZ', 'LUZ ANDREA BERMUDEZ SANDO

In [7]:
df = df[~df["candidate"].isin(["BLANK", "SPOILT", "TOTALES", np.nan])]

In [8]:
df = df.dropna(axis=1, how="all")

In [9]:
df_filtered = df.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
df_filtered["rate"] = df_filtered.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
df_filtered = df_filtered.reset_index()
df_filtered["rank"] = df_filtered.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)
df_filtered["flag_candidates"] = 1
df_filtered.head()

,polling_id,candidate,value,rate,rank,flag_candidates
0,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ABRAHAM GONZALO LARRONDO VEGA,4.0,0.030534,11,1
1,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ANGELICA SAN CRISTOBAL SILVA,3.0,0.022901,15,1
2,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ANTARIS CATALINA VARELA COMPAGNON,8.0,0.061069,5,1
3,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,DANIELA DRESDNER VICENCIO,16.0,0.122137,2,1
4,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,DARWIN RODRIGO ACUÑA BECERRA,2.0,0.015267,19,1


In [10]:
data = pd.concat([
    df_filtered,
    df_abstentions,
    df_spoilt
])

data.head()

,polling_id,candidate,value,rate,rank,flag_candidates
0,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ABRAHAM GONZALO LARRONDO VEGA,4.0,0.030534,11.0,1
1,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ANGELICA SAN CRISTOBAL SILVA,3.0,0.022901,15.0,1
2,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ANTARIS CATALINA VARELA COMPAGNON,8.0,0.061069,5.0,1
3,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,DANIELA DRESDNER VICENCIO,16.0,0.122137,2.0,1
4,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,DARWIN RODRIGO ACUÑA BECERRA,2.0,0.015267,19.0,1


In [11]:
data.groupby("candidate").agg({"value": "sum"})

,value
candidate,
ABRAHAM GONZALO LARRONDO VEGA,6078.0
ABSTENTION,5654910.0
ADRIANA MUÑOZ DALBORA,18852.0
ALBERTO PIZARRO CHAÑILAO,8575.0
ALEJANDRA SEPULVEDA ORBENES,120355.0
...,...
VARINIA YUSETH ARAVENA PEREZ,5627.0
VERONICA PILAR PARDO LAGOS,26107.0
VICTORIA ORTIZ VERA,4289.0


In [12]:
print(df_filtered.shape)
df_filtered.head()

(1087101, 6)


,polling_id,candidate,value,rate,rank,flag_candidates
0,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ABRAHAM GONZALO LARRONDO VEGA,4.0,0.030534,11,1
1,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ANGELICA SAN CRISTOBAL SILVA,3.0,0.022901,15,1
2,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,ANTARIS CATALINA VARELA COMPAGNON,8.0,0.061069,5,1
3,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,DANIELA DRESDNER VICENCIO,16.0,0.122137,2,1
4,10-20-CHIGUAYANTE-CHIGUAYANTE-COLEGIO AURORA D...,DARWIN RODRIGO ACUÑA BECERRA,2.0,0.015267,19,1


In [21]:
if "party" in list(df):
    data = pd.merge(
        data,
        df[["party", "candidate"]].drop_duplicates(),
        on="candidate"
    )

data.to_csv(f"data_output/Chile/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [14]:
list(data)

['polling_id', 'candidate', 'value', 'rate', 'rank', 'flag_candidates']

In [15]:
# df[["polling_id", "Región", "Provincia", "Circ.Senatorial", "Distrito", "Comuna", "Circ.Electoral", "Local",
#         "Mesa", "Tipo mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

In [17]:
if year == 2021:

    df_location = df[["polling_id", "nro. región", "región", "provincia", "circ. senatorial", "distrito", "comuna", "circ. electoral", "local",
        "mesa", "tipo mesa", "mesas fusionadas", "electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "nro. región": "region_id",
        "región": "region",
        "provincia": "province",
        "circ. senatorial": "senate_district",
        "distrito": "district",
        "comuna": "commune",
        "circ. electoral": "electoral_circ",
        "local": "polling_place",
        "mesa": "polling_table",
        "tipo mesa": "polling_type",
        "mesas fusionadas": "mixed_polling",
        "electores": "voters"
    })
    
    for col in ["region_id", "senate_district", "district", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)
    
elif year == 2017:
    df_location = df[["polling_id", "región", "provincia", "circ.senatorial", "distrito", "comuna", "circ. electoral", "local",
        "mesa", "tipo mesa", "mesas fusionadas", "electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "región": "region_id",
        "provincia": "province",
        "circ.senatorial": "senate_district",
        "distrito": "district",
        "comuna": "commune",
        "circ.electoral": "electoral_circ",
        "local": "polling_place",
        "mesa": "polling_table",
        "tipo mesa": "polling_type",
        "mesas fusionadas": "mixed_polling",
        "electores": "voters"
    })
    
    for col in ["region_id", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)

elif year == 2013:
    df_location = df[["polling_id", "region_id", "provincia", "circ. senatorial", "distrito", "comuna", "circ. electoral", 
        "nro. mesa", "tipo mesa"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "provincia": "province",
        "circ. senatorial": "senate_district",
        "distrito": "district",
        "comuna": "commune",
        "circ. electoral": "electoral_circ",
        "nro. mesa": "polling_table",
        "tipo mesa": "polling_type"
    })
    
    for col in ["region_id", "polling_table"]:
        df_location[col] = df_location[col].astype(int)

df_location.to_csv(f"data_output/Chile/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)